<a href="https://colab.research.google.com/github/Ferstuque/VoiceControl/blob/main/Text_to_audio___audio_to_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Audio to Text and Text to Audio Notebook

This notebook is based on the Carleslc Notebook. You can find the original and complete version at [LINK](https://github.com/Carleslc/AudioToText?tab=readme-ov-file)

⚠️ For faster processing, consider running this Notebook on a GPU.

[Whisper by OpenAI](https://github.com/openai/whisper)

Let's start by installing the necessary dependencies for this notebook.

In [ ]:
#@title { display-mode: "form" }
import subprocess

from sys import platform as sys_platform

status, ffmpeg_version = subprocess.getstatusoutput("ffmpeg -version")

if status != 0:
  from platform import platform

  if sys_platform == 'linux' and 'ubuntu' in platform().lower():
    !apt install ffmpeg
  else:
    print("Install ffmpeg: https://ffmpeg.org/download.html")
else:
  print(ffmpeg_version.split('\n')[0])

  NO_ROOT_WARNING = '|& grep -v \"WARNING: Running pip as the \'root\' user"' # running in Colab

  !pip install --no-warn-script-location --user --upgrade pip {NO_ROOT_WARNING}
  !pip install --root-user-action=ignore git+https://github.com/openai/whisper.git@v20231117 openai==1.9.0 numpy scipy deepl pydub cohere ffmpeg-python torch==2.1.0 tensorflow-probability==0.23.0 typing-extensions==4.9.0
  !pip install gtts

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  Cloning https://github.com/openai/whisper.git (to revision v20231117) to /tmp/pip-req-build-vc3sayfb
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-vc3sayfb
  Running command git checkout -q e58f28804528831904c3b6f2c0e473f346223433
  Resolved https://github.com/openai/whisper.git to commit e58f28804528831904c3b6f2c0e473f346223433
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


## Text to audio

In [ ]:
#@title Click ▶️ button below and write your text
from gtts import gTTS
input_text = input()
text_to_say = input_text

Hallo, wie geht es dir?


In [ ]:
#@title Now, select your language before run this cell

language = "de" #@param ["af", "sq", "am", "ar", "hy", "as", "az", "ba", "eu", "be", "bn", "bs", "br", "bg", "my", "es", "ca", "zh", "hr", "cs", "da", "nl", "en", "et", "fo", "fi", "nl", "fr", "gl", "ka", "de", "el", "gu", "ht", "ht", "ha", "ha", "he", "hi", "hu", "is", "id", "it", "ja", "jv", "kn", "kk", "km", "ko", "lo", "la", "lv", "lb", "ln", "lt", "lb", "mk", "mg", "ms", "ml", "mt", "mi", "mr", "ro", "ro", "mn", "my", "ne", "no", "nn", "oc", "pa", "ps", "fa", "pl", "pt", "pa", "ps", "ro", "ru", "sa", "sr", "sn", "sd", "si", "si", "sk", "sl", "so", "es", "su", "sw", "sv", "tl", "tg", "ta", "tt", "te", "th", "bo", "tr", "tk", "uk", "ur", "uz", "ca", "vi", "cy", "yi", "yo"]

gtts_object = gTTS(text = text_to_say,
                  lang = language,
                  slow = False)

gtts_object.save("/content/gtts.wav")

from IPython.display import Audio

Audio("/content/gtts.wav")

## Audio to Text

### In this session an AI can detect your language to trancribe it to text.


**Task:** Choose an action, Transcribe or Translate your previous audio recording.

**Use Module:** If the execution time is excessive, consider using a smaller model in `use_model`. This may result in a slight decrease in accuracy, but can improve performance. Alternatively, you can utilize the OpenAI API.

**Language:** Choose your language from the list or select "Auto-Detect" to have your recorded file processed automatically.

**Prompt:** You can add an optional initial prompt to provide context about the audio or to guide the AI towards a specific writing style. Refer to the prompting [guide](https://platform.openai.com/docs/guides/speech-to-text/prompting) for more information.




In [ ]:
#@title Record your audio and save it
# This cell code is a slightly modified version of DotCSV code in the following Colab along with other references:
# https://colab.research.google.com/drive/1CvvYPAFemIZdSOt9fhN541esSlZR7Ic6?usp=sharing

try:
  import io
  import os
  import ffmpeg
  import numpy as np

  # Only available in Google Colab
  from google.colab.output import eval_js

  from IPython.display import HTML, Audio
  from scipy.io.wavfile import write, read as wav_read
  from base64 import b64decode
  from os.path import isfile

  AUDIO_HTML = """
  <script>
  var my_div = document.createElement("DIV");
  var my_p = document.createElement("P");
  var my_btn = document.createElement("BUTTON");
  var t = document.createTextNode("Starting recording...");

  my_btn.appendChild(t);
  my_div.appendChild(my_btn);
  document.body.appendChild(my_div);

  var base64data = 0;
  var reader;
  var recorder, gumStream;
  var recordButton = my_btn;

  var handleSuccess = function(stream) {
    gumStream = stream;
    var options = {
      bitsPerSecond: 16000,
      mimeType : 'audio/webm;codecs=opus' //codecs=pcm
    };
    recorder = new MediaRecorder(stream, options);
    //recorder = new MediaRecorder(stream);

    recorder.ondataavailable = function(e) {
      var url = URL.createObjectURL(e.data);
      var preview = document.createElement('audio');
      preview.controls = true;
      preview.src = url;
      document.body.appendChild(preview);

      reader = new FileReader();
      reader.readAsDataURL(e.data);
      reader.onloadend = function() {
        base64data = reader.result;
        //console.log("reader.onloadend: " + base64data);
      }
    };
    recorder.start();
    recordButton.innerText = "🔴 Recording... press to STOP";
  };

  navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);

  function toggleRecording() {
    if (recorder && recorder.state == "recording") {
        recorder.stop();
        gumStream.getAudioTracks()[0].stop();
        recordButton.innerText = "Saving the recording... please wait!";
    }
  }

  // https://stackoverflow.com/a/951057
  function sleep(ms) {
    return new Promise(resolve => setTimeout(resolve, ms));
  }

  var data = new Promise(resolve => {
    recordButton.onclick = () => {
      toggleRecording();

      sleep(2000).then(() => {
        // wait 2000ms for the data to be available...
        //console.log("resolve data: " + base64data);
        resolve(base64data.toString());
      });
    }
  });

  function doneRecording(recording_file) {
    my_div.removeChild(recordButton);
    my_p.innerText = recording_file;
    my_div.appendChild(my_p);
  }

  </script>
  """

  def get_audio():
    display(HTML(AUDIO_HTML))
    data = eval_js("data")
    binary = b64decode(data.split(',')[1])

    process = (ffmpeg
      .input('pipe:0')
      .output('pipe:1', format='wav')
      .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
    )
    output, err = process.communicate(input=binary)

    riff_chunk_size = len(output) - 8
    # Break up the chunk size into four bytes, held in b.
    q = riff_chunk_size
    b = []
    for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

    # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
    riff = output[:4] + bytes(b) + output[8:]

    sr, audio = wav_read(io.BytesIO(riff))

    return audio, sr

  recording_file = "recording.wav" #@param {type:"string"}

  if isfile(recording_file):
    print(f"{recording_file} File name already exists. You can either create another recording with a different name, or delete the existing file first.")
  else:
    # record microphone
    audio, sr = get_audio()

    # write recording
    write(recording_file, sr, audio)

    eval_js(f'doneRecording("{recording_file}")')
except ImportError:
  print("Recording only available in Google Colab")

In [ ]:
import os, subprocess

import whisper
from whisper.utils import format_timestamp, get_writer, WriteTXT

import numpy as np

try:
  import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
  pass

import torch

import math

from openai import OpenAI

# select task

task = "Translate to English" #@param ["Transcribe", "Translate to English"]

task = "transcribe" if task == "Transcribe" else "translate"

# select audio file

audio_file = recording_file

audio_files = list(map(lambda audio_path: audio_path.strip(), audio_file.split(',')))

for audio_path in audio_files:
  if not os.path.isfile(audio_path):
    raise FileNotFoundError(audio_path)

# set model

use_model = "small" #@param ["tiny", "base", "small", "medium", "large-v1", "large-v2"]

# select language

language = "Auto-Detect" #@param ["Auto-Detect", "Afrikaans", "Albanian", "Amharic", "Arabic", "Armenian", "Assamese", "Azerbaijani", "Bashkir", "Basque", "Belarusian", "Bengali", "Bosnian", "Breton", "Bulgarian", "Burmese", "Castilian", "Catalan", "Chinese", "Croatian", "Czech", "Danish", "Dutch", "English", "Estonian", "Faroese", "Finnish", "Flemish", "French", "Galician", "Georgian", "German", "Greek", "Gujarati", "Haitian", "Haitian Creole", "Hausa", "Hawaiian", "Hebrew", "Hindi", "Hungarian", "Icelandic", "Indonesian", "Italian", "Japanese", "Javanese", "Kannada", "Kazakh", "Khmer", "Korean", "Lao", "Latin", "Latvian", "Letzeburgesch", "Lingala", "Lithuanian", "Luxembourgish", "Macedonian", "Malagasy", "Malay", "Malayalam", "Maltese", "Maori", "Marathi", "Moldavian", "Moldovan", "Mongolian", "Myanmar", "Nepali", "Norwegian", "Nynorsk", "Occitan", "Panjabi", "Pashto", "Persian", "Polish", "Portuguese", "Punjabi", "Pushto", "Romanian", "Russian", "Sanskrit", "Serbian", "Shona", "Sindhi", "Sinhala", "Sinhalese", "Slovak", "Slovenian", "Somali", "Spanish", "Sundanese", "Swahili", "Swedish", "Tagalog", "Tajik", "Tamil", "Tatar", "Telugu", "Thai", "Tibetan", "Turkish", "Turkmen", "Ukrainian", "Urdu", "Uzbek", "Valencian", "Vietnamese", "Welsh", "Yiddish", "Yoruba"]

# other parameters

prompt = "" #@param {type:"string"}

coherence_preference = "More coherence, but may repeat text" #@param ["More coherence, but may repeat text", "Less repetitions, but may have less coherence"]

api_key = ''

# detect device

if api_key:
  print("Using API")

  from pydub import AudioSegment
  from pydub.silence import split_on_silence
else:
  DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

  print(f"Using {'GPU' if DEVICE == 'cuda' else 'CPU ⚠️'}")

  # https://medium.com/analytics-vidhya/the-google-colab-system-specification-check-69d159597417
  if DEVICE == "cuda":
    !nvidia-smi -L
  else:
    if sys_platform == 'linux':
      !lscpu | grep "Model name" | awk '{$1=$1};1'

    print("Not using GPU can result in a very slow execution")
    print("Ensure Hardware accelerator by GPU is enabled in Google Colab: Runtime > Change runtime type")

    if use_model not in ['tiny', 'base', 'small']:
      print("You may also want to try a smaller model (tiny, base, small)")

# display language

WHISPER_LANGUAGES = [k.title() for k in whisper.tokenizer.TO_LANGUAGE_CODE.keys()]

if language == "Auto-Detect":
  language = "detect"

if language and language != "detect" and language not in WHISPER_LANGUAGES:
  print(f"\nLanguage '{language}' is invalid")
  language = "detect"

if language and language != "detect":
  print(f"\nLanguage: {language}")

# load model

if api_key:
  print()
else:
  MODELS_WITH_ENGLISH_VERSION = ["tiny", "base", "small", "medium"]

  if language == "English" and use_model in MODELS_WITH_ENGLISH_VERSION:
    use_model += ".en"

  print(f"\nLoading {use_model} model... {os.path.expanduser(f'~/.cache/whisper/{use_model}.pt')}")

  model = whisper.load_model(use_model, device=DEVICE)

  print(
      f"Model {use_model} is {'multilingual' if model.is_multilingual else 'English-only'} "
      f"and has {sum(np.prod(p.shape) for p in model.parameters()):,d} parameters.\n"
  )

# set options

## https://github.com/openai/whisper/blob/v20231117/whisper/transcribe.py#L37
## https://github.com/openai/whisper/blob/v20231117/whisper/decoding.py#L81
options = {
    'task': task,
    'verbose': True,
    'fp16': True,
    'best_of': 5,
    'beam_size': 5,
    'patience': None,
    'length_penalty': None,
    'suppress_tokens': '-1',
    'temperature': (0.0, 0.2, 0.4, 0.6, 0.8, 1.0), # float or tuple
    'condition_on_previous_text': coherence_preference == "More coherence, but may repeat text",
    'initial_prompt': prompt or None,
    'word_timestamps': False,
}

if api_key:
  api_client = OpenAI(api_key=api_key)

  api_supported_formats = ['mp3', 'mp4', 'mpeg', 'mpga', 'm4a', 'wav', 'webm']
  api_max_bytes = 25 * 1024 * 1024 # 25 MB

  api_transcribe = api_client.audio.transcriptions if task == 'transcribe' else api_client.audio.translations
  api_transcribe = api_transcribe.create

  api_model = 'whisper-1' # large-v2

  # https://platform.openai.com/docs/api-reference/audio?lang=python
  api_options = {
    'response_format': 'verbose_json',
  }

  if prompt:
    api_options['prompt'] = prompt

  api_temperature = options['temperature'][0] if isinstance(options['temperature'], (tuple, list)) else options['temperature']

  if isinstance(api_temperature, (float, int)):
    api_options['temperature'] = api_temperature
  else:
    raise ValueError("Invalid temperature type, it must be a float or a tuple of floats")
elif DEVICE == 'cpu':
  options['fp16'] = False
  torch.set_num_threads(os.cpu_count())

# execute task
# !whisper "{audio_file}" --task {task} --model {use_model} --output_dir {output_dir} --device {DEVICE} --verbose {options['verbose']}

if task == "translate":
  print("-- TRANSLATE TO ENGLISH --")
else:
  print("-- TRANSCRIPTION --")

results = {} # audio_path to result

for audio_path in audio_files:
  print(f"\nProcessing: {audio_path}\n")

  # detect language
  detect_language = not language or language == "detect"

  if not detect_language:
    options['language'] = language
    source_language_code = whisper.tokenizer.TO_LANGUAGE_CODE.get(language.lower())
  elif not api_key:
    # load audio and pad/trim it to fit 30 seconds
    audio = whisper.load_audio(audio_path)
    audio = whisper.pad_or_trim(audio)

    # make log-Mel spectrogram and move to the same device as the model
    mel = whisper.log_mel_spectrogram(audio).to(model.device)

    # detect the spoken language
    _, probs = model.detect_language(mel)

    source_language_code = max(probs, key=probs.get)
    options['language'] = whisper.tokenizer.LANGUAGES[source_language_code].title()

    print(f"Detected language: {options['language']}\n")

  # transcribe
  if api_key:
    # API
    if task == "transcribe" and not detect_language:
      api_options['language'] = source_language_code

    source_audio_name_path, source_audio_ext = os.path.splitext(audio_path)
    source_audio_ext = source_audio_ext[1:]

    if source_audio_ext in api_supported_formats:
      api_audio_path = audio_path
      api_audio_ext = source_audio_ext
    else:
      ## convert audio file to a supported format
      if options['verbose']:
        print(f"API supported formats: {','.join(api_supported_formats)}")
        print(f"Converting {source_audio_ext} audio to a supported format...")

      api_audio_ext = 'mp3'

      api_audio_path = f'{source_audio_name_path}.{api_audio_ext}'

      subprocess.run(['ffmpeg', '-i', audio_path, api_audio_path], check=True, capture_output=True)

      if options['verbose']:
        print(api_audio_path, end='\n\n')

    ## split audio file in chunks
    api_audio_chunks = []

    audio_bytes = os.path.getsize(api_audio_path)

    if audio_bytes >= api_max_bytes:
      if options['verbose']:
        print(f"Audio exceeds API maximum allowed file size.\nSplitting audio in chunks...")

      audio_segment_file = AudioSegment.from_file(api_audio_path, api_audio_ext)

      min_chunks = math.ceil(audio_bytes / (api_max_bytes / 2))

      # print(f"Min chunks: {min_chunks}")

      max_chunk_milliseconds = int(len(audio_segment_file) // min_chunks)

      # print(f"Max chunk milliseconds: {max_chunk_milliseconds}")

      def add_chunk(api_audio_chunk):
        api_audio_chunk_path = f"{source_audio_name_path}_{len(api_audio_chunks) + 1}.{api_audio_ext}"
        api_audio_chunk.export(api_audio_chunk_path, format=api_audio_ext)
        api_audio_chunks.append(api_audio_chunk_path)

      def raw_split(big_chunk):
        subchunks = math.ceil(len(big_chunk) / max_chunk_milliseconds)

        for subchunk_i in range(subchunks):
          chunk_start = max_chunk_milliseconds * subchunk_i
          chunk_end = min(max_chunk_milliseconds * (subchunk_i + 1), len(big_chunk))
          add_chunk(big_chunk[chunk_start:chunk_end])

      non_silent_chunks = split_on_silence(audio_segment_file,
                                           seek_step=5, # ms
                                           min_silence_len=1250, # ms
                                           silence_thresh=-25, # dB
                                           keep_silence=True) # needed to aggregate timestamps

      # print(f"Non silent chunks: {len(non_silent_chunks)}")

      current_chunk = non_silent_chunks[0] if non_silent_chunks else audio_segment_file

      for next_chunk in non_silent_chunks[1:]:
        if len(current_chunk) > max_chunk_milliseconds:
          raw_split(current_chunk)
          current_chunk = next_chunk
        elif len(current_chunk) + len(next_chunk) <= max_chunk_milliseconds:
          current_chunk += next_chunk
        else:
          add_chunk(current_chunk)
          current_chunk = next_chunk

      if len(current_chunk) > max_chunk_milliseconds:
        raw_split(current_chunk)
      else:
        add_chunk(current_chunk)

      if options['verbose']:
        print(f'Total chunks: {len(api_audio_chunks)}\n')
    else:
      api_audio_chunks.append(api_audio_path)

    ## process chunks
    result = None

    for api_audio_chunk_path in api_audio_chunks:
      ## API request
      with open(api_audio_chunk_path, 'rb') as api_audio_file:
        api_result = api_transcribe(model=api_model, file=api_audio_file, **api_options)
        api_result = api_result.model_dump() # to dict

      api_segments = api_result['segments']

      if result:
        ## update timestamps
        last_segment_timestamp = result['segments'][-1]['end'] if result['segments'] else 0

        for segment in api_segments:
          segment['start'] += last_segment_timestamp
          segment['end'] += last_segment_timestamp

        ## append new segments
        result['segments'].extend(api_segments)

        if 'duration' in result:
          result['duration'] += api_result.get('duration', 0)
      else:
        ## first request
        result = api_result

        if detect_language:
          print(f"Detected language: {result['language'].title()}\n")

      ## display segments
      if options['verbose']:
        for segment in api_segments:
          print(f"[{format_timestamp(segment['start'])} --> {format_timestamp(segment['end'])}] {segment['text']}")
  else:
    # Open-Source
    result = whisper.transcribe(model, audio_path, **options)

  # fix results formatting
  for segment in result['segments']:
    segment['text'] = segment['text'].strip()

  result['text'] = '\n'.join(map(lambda segment: segment['text'], result['segments']))

  # set results for this audio file
  results[audio_path] = result

Using CPU ⚠️
Model name: Intel(R) Xeon(R) CPU @ 2.20GHz
Not using GPU can result in a very slow execution
Ensure Hardware accelerator by GPU is enabled in Google Colab: Runtime > Change runtime type

Loading small model... /root/.cache/whisper/small.pt
Model small is multilingual and has 240,582,912 parameters.

-- TRANSLATE TO ENGLISH --

Processing: recording.wav

Detected language: German

[00:00.000 --> 00:02.000]  How old are you?
